## predict__ROC_trainSet

15/06/16 by greghor


on commence par regarder les documents ou l on observe le mot center
utilise les fichiers .html
integre corrections typographiques pour coller à la solution
integre ROC à partir des currencies

In [1]:
# SYSTEM
from os import listdir
from PyCommonFun import *

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *

#GENERAL
import numpy as np
import pandas as pd
import pickle
import string

# data from Albert
from code.dataExtract import data
import code.dataExtract as de
from code import Text_reader as tr
# Read the data
d = data(folder='../../int_test/') # training set with all the labels
dataPath='../../int_test/'



The PythonMagick package is missing, get_text_adv function can not be used 
The PyPDF2 package is missing, get_text_adv function can not be used 
The html2text package is missing, get_text_adv and get_text functions can not be used.
Please consider the instalation of this package !


In [2]:
# fonctions used in the notebook

def getContext(s,pattern,context=list(),width=100):
    
    
    loc=s.find(pattern)
    
    if loc == -1:
        return context
    elif loc-width < 0:
        context+=s[:loc+width]
        return getContext(s[loc+len(pattern):],pattern,context,width)
    elif loc + width > len(s):
        context+=s[loc-width:]
        return getContext(s[loc+len(pattern):],pattern,context,width)
    else:
        context+=s[loc-width:loc+width]
        return getContext(s[loc+len(pattern):],pattern,context,width)


    
def useSolutionTypo(s):
    s=s.lower()
    if s=="sao paulo":
        s="sao-paulo"
    elif s=='target 1' or s =='target 2' or s == 'target' or s=='target1':
        s='target holidays'
    return s



def document_from_hmtl(doc_path):
    with open(doc_path) as html_file:
        full_text = html_file.read()
        body = re.findall(r'<body[^>]*?>(.*?)</body>', full_text)
        cleantext = re.sub('<\/?span[^>]*>', ' ', body[0])
        clean_text = re.sub('<.*?>', ' ', cleantext)
        clean_text = clean_text.lower()
        clean_text = clean_text.lower()
        clean_text = re.sub('0.01', '1', clean_text)
        clean_text = re.sub(',', '', clean_text)
        clean_text = re.sub(r'[^a-z0-9]', ' ', clean_text)
        clean_text = re.sub('\s+', ' ', clean_text)
        return clean_text

def removeUnwantedCity(city, unwantedCity):
    #
    assert isinstance(city,list) or isinstance(city,np.ndarray), "city should be an np.array or a list"
    assert isinstance(unwantedCity,list),  "unwanted city should be a list"
        
    for gi in range(len(city)):
        for unwanted in unwantedCity:
            if '|' + unwanted in city[gi]:
                city[gi]=city[gi].replace('|'+ unwanted,'')
            elif unwanted + '|' in city[gi]:
                city[gi]=city[gi].replace(unwanted + '|','')
            elif unwanted in city[gi]:
                city[gi]=city[gi].replace(unwanted,'')
    return city
                

In [3]:
# get the solution
ROCTmp=pd.read_csv('ROC_fullList.csv',delimiter=";")
cities_groundTruth=np.load('../currencyExtraction/predicted_opCurr_dict.npy').item()
ROC_list=['target 1','target 2','target'] # add target 1 and target 2
# lowerise ROC and remove ponctuation
for roc in ROCTmp['CIT_NM'].values:
    roc=str(roc).lower().translate(None, string.punctuation)
    ROC_list.append(roc.rstrip())

In [4]:
# link roc and operationnal curency
opCurrency=np.load('../currencyExtraction/predicted_opCurr_dict.npy').item()
opCurr_df=pd.DataFrame(opCurrency.items(),columns=['ISIN','OpCurrency'])
link_roc_opCurr=pd.read_csv('currency_ROC_fullList.csv',delimiter=";")
# merge with labels on operationnal currency
roc_fromCurrency=pd.merge(left=link_roc_opCurr,right=opCurr_df,left_on='Currency Code', right_on='OpCurrency')
# keep only currency code and default ROC columns
col=["ISIN","OpCurrency","Default ROC",]
roc_fromCurrency=roc_fromCurrency[col]
roc_fromCurrency.rename(columns={'Default ROC':'currency_ROC'},inplace=True)
len(roc_fromCurrency)

426

In [5]:
gf=0


context_pat=dict()
text=dict()
htmlFiles=tr.get_files(dataPath + '/html')



for isin in d.docid.keys()[:]:
        
    text[isin]=document_from_hmtl(htmlFiles[d.docid[isin][0]])
    context_pat[isin]=''

    pattern=["Business Day Centre","Financial Centre","Business Centre","Currency Business Day",'Business Day']
    
    ## recupere le contexte si patterns, prend l ensemble du text sinon
    
    context_pat[isin]+=text[isin]
    
    """  
    for pat in pattern:
        context_pat[isin]+=getContext(text[isin],pat.lower(),'',150)
        # il faut impérativement passer, un troisieme argument vide pour que le code 
        # fonctionne mais je ne comprends pas bien pourquoi
            
    if len(context_pat[isin])==0:
        context_pat[isin]+=text[isin]"""
            
    gf+=1
    progression=float(gf)/len(d.docid.keys())*100
    if gf%100==0:
        print float(gf)/len(d.docid.keys())
       


0.234741784038
0.469483568075
0.704225352113
0.93896713615


In [6]:
len(context_pat.keys())

426

#### predict


In [14]:
# scikit impory
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from time import time

text_clf=joblib.load('sgd_model/roc_70Trainpipeline.pkl')


In [16]:
# create the conext and load the model
context_df=pd.DataFrame(context_pat.items(),columns=['isin','context'])
context=context_df.as_matrix(columns=['context']).ravel()
# load the model
text_clf=joblib.load('sgd_model/roc_70Trainpipeline.pkl')
#predict
predicted_roc=text_clf.predict(context)


In [22]:
predicted_roc_dict=dict()

gi=0
for isin in context_df['isin'].values:
    predicted_roc_dict[isin]=predicted_roc[gi]
    gi+=1

In [26]:
pickle.dump(predicted_roc_dict,open('predicted_roc_dict.pickle','w'))